# Sistema usuario-usuario

In [11]:
import pandas as pd, numpy as np, time, os
from IPython.display import display
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD, NormalPredictor
import pickle
from surprise.model_selection import cross_validate, train_test_split, KFold
from surprise.prediction_algorithms.knns import *
from surprise.accuracy import rmse, mae
from collections import defaultdict


In [9]:
def cargar_archivo(archivo, pickle, columnas, fechas, sep = '\t'):

    try:
        t1 = time.clock()
        print('Cargando el pickle (si existe)...')
        tabla = pd.read_pickle(pickle)
        t2 = time.clock()
        print('Leer la tabla DATASET toma {:.2f} segundos'.format(t2 - t1))

    except FileNotFoundError:
        
        print('El Pickle NO existe. Creando el Pickle...')
        t1 = time.clock()
        tabla = pd.read_csv(archivo, sep = sep, names = columnas)
        t2 = time.clock()

        print('Leer la tabla toma {:.2f} segundos'.format(t2 - t1))

        tabla.to_pickle(pickle)

    return tabla

def cargar_dataset():
    fechas = ['TIMESTAMP']
    archivo = os.path.join('.', 'toronto_reviews_plus90.csv')
    pickle = './dataset_reviews.pickle'
    columnas = ['review', 'stars', 'user', 'business', 'useful', 'cool', 'funny']
    return cargar_archivo(archivo, pickle, columnas, fechas)

Cargue del dataset (cuando no existe el _Pickle_, toma cerca de tres minutos; cuando sí existe, toma cerca de 12 segundos):

In [10]:
dataset = cargar_dataset()

./toronto_reviews_plus90.csv
Cargando el pickle (si existe)...
El Pickle NO existe. Creando el Pickle...
Leer la tabla toma 0.11 segundos


In [15]:
try:

    t1 = time.clock()
    print('Cargando el pickle (si existe)...')
    dataset_2 = pickle.load(open('./toronto_reviews_array.pickle', 'rb'))
    t2 = time.clock()
    print('Leer la tabla DATASET toma {:.2f} segundos'.format(t2 - t1))

except FileNotFoundError:

    print('El Pickle NO existe. Creando el Pickle...')
    t1 = time.clock()
    dataset_2 = dataset[['user', 'business', 'stars']].pivot_table(index = 'user', columns = 'business', aggfunc = np.sum, fill_value = 0)
    t2 = time.clock()

    print('Leer la tabla toma {:.2f} segundos'.format(t2 - t1))

    pickle.dump(dataset_2, open('./toronto_reviews_array.pickle', 'wb'))

#dataset[['USERID', 'ARTIST_NAME', 'VALOR']].pivot_table(index = 'USERID', columns = 'ARTIST_NAME', aggfunc = lambda x: len(x.unique()), fill_value = 0)
display(dataset_2)

Cargando el pickle (si existe)...
Leer la tabla DATASET toma 0.05 segundos


stars                         \
business               --DaPTJW3-tB1vP-PfdTEg --SrzpvFLwP_YFwB_Cetow   
user                                                                   
--Qh8yKWAvIP4V4K8ZPfHA                      0                      0   
-KVxkJDSTjtPGsamMDG92Q                      0                      0   
-RCD8F7qbsLfzT3k1HtMxg                      0                      0   
-_2h2cJlBOWAYrfplMU-Cg                      0                      0   
-d2daWmftYumOaYpbD5D8Q                      0                      0   
-od707p4FHGul0gte29AoQ                      0                      0   
-pXs08gJq9ExIk275YLvPg                      0                      0   
-qr-SDo-IHa5-YISjtmesQ                      0                      0   
-tdsrQ3QIkGmmP2n6-DTeg                      0                      0   
02AOwJB1g9ETIDGZznyuAw                      0                      0   
0BBUmH7Krcax1RZgbH4fSA                      0                      0   
0BaJ43WuBnP-G6fmstEmNQ                      0                      0   
0IwZb1ePNpSVYcCdMtGoQw                      0                      0   
0J9mtVJ4_QGsXYVK8qJzOQ                      0                      0   
0LMHrFTQtUoY42AZl1MH6g                      0                      0   
0gtEsazPeHr5rj2IAsZePg                      0                      0   
0l3x4EgoWVYdla3CPpAKag                      0                      0   
0rDG0uZHpqxJbdu-FJ3IgA                      0                      0   
0uNxhZAFbkalQImzJ6UDUA                      0                      0   
0uacsMYppmm3Kl1bpWIOxQ                      0                      0   
0w03V5AjlO_EHnRRorlLFQ                      0                      0   
0xRAAStEi_dBFLD-1xSMHQ                      0                      0   
13xeC5yKJO4BDyQOI3xxDQ                      0                      0   
1FB1Vc4zne5AJAliqdMeWA                      0                      0   
1HFkcIGKY7-e9vCsc8PX5g                      0                      0   
1fNQRju9gmoCEvbPQBSo7w                      4                      0   
1ickSuyOWWKWdQpauKmkEw                      0                      0   
1kNsEAhGU8d8xugMuXJGFA                      0                      0   
1tVRQliYQ_Xw4YGy52njwg                      0                      0   
23o7tyUGlC6FCDVcyqLeFA                      0                      0   
...                                       ...                    ...   
wdeWt5VqTW26PAeQsVg73g                      0                      0   
wskTNZjwiqMJOvM_-XAQZw                      0                      0   
x2o4DyS9DUUTD2S0vwlDCQ                      0                      0   
xDed_RrQv7xuwOp9hESt9Q                      0                      0   
xHMq2fBArXBp881TVlRt5g                      0                      0   
xVV86BToDOQGxE1gt9MMvg                      0                      0   
xXyZPAbfQzYTCig7jnZO0A                      0                      0   
xYciRtVZ1PW4IxSX4oJ1aw                      0                      0   
xaCU92mNCFAJ9BdtfoP7dA                      0                      0   
xdYAjnWeAHQfcLH7c2l4OQ                      0                      0   
xsBjO0GTAvZK8i2A8_z20g                      0                      0   
y0Fto7ianqYeO9z-nvxVsQ                      0                      0   
y0ZU1w6YY4W-KtMeRXSYLg                      0                      0   
y3MNMa0SG_cHC1cwmmntpQ                      0                      0   
yHakD_sjdUAzLQWLslmUfg                      0                      0   
yHykh6VFpsH_V62bRaq3FQ                      0                      0   
yK6wyLZ2I66B5-MJcS88xg                      0                      0   
yT_QCcnq-QGipWWuzIpvtw                      0                      0   
yaWnpONXlvdXbi36aKcznw                      0                      0   
z66KKknqWzFBWZb0GbEi2g                      0                      0   
z6gseuVl0cR7tRLQa_DXuQ                      0                      0   
z9w399cBpCAKXhH_JA1AtQ                      0                      0   
zCnJQKvZ

Convierto el dataset a un dataset de surprise.

In [16]:
surprise_ds = dataset_2.stack()
surprise_ds.reset_index(level = [0, 1], inplace = True)
surprise_ds = surprise_ds[surprise_ds.stars != 0]

In [17]:
kf = KFold(n_splits = 2)

In [18]:
reader = Reader(rating_scale = (1, 5))

In [23]:
data = Dataset.load_from_df(surprise_ds[['user', 'business', 'stars']], reader)

for trainset, testset in kf.split(data):
    sim_options = {'name': 'msd', 'user_based': False, 'min_k': 10, 'k': 15}
    algo = KNNBasic(sim_options = sim_options)
    algo.fit(trainset)
    
    test_set = testset

    predictions = algo.test(testset)
    rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0469
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0479


In [62]:
data = Dataset.load_from_df(surprise_ds[['user', 'business', 'stars']], reader)
train_data = data.build_full_trainset()
sim_options = {'name': 'msd', 'user_based': False, 'min_k': 10, 'k': 15}
algo = KNNBasic(sim_options = sim_options)
algo.fit(train_data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [63]:
test = [x for x in train_data.build_testset() if x[0] == "--Qh8yKWAvIP4V4K8ZPfHA"]

In [64]:
len(test)

383

In [65]:
predictions = algo.test(test)

In [66]:
def get_top_n(predictions, n = 10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n = 10)

top_n = dict(top_n)
top_n['--Qh8yKWAvIP4V4K8ZPfHA']

[('HPr3UUlwzfmXpwhUSg6_-g', 4.620689655172414),
 ('Jn17OhYIWrSlXHI3bpZsLQ', 4.617142857142857),
 ('GdiKn-JExlIHCQUDxQ9xgQ', 4.613636363636363),
 ('-PLXDic2ALf15NLKHFHx3A', 4.605714285714285),
 ('fubrLSuFj9IHh_Jk9sTI8w', 4.605714285714285),
 ('FEE0dl5XqLS0RHKijqt-Lw', 4.601156069364162),
 ('YY23OABxBKOryNqtmMOKjA', 4.592178770949721),
 ('M30Ndb5zaFyU0m-qVLngiw', 4.586206896551723),
 ('uMOOP1DtKAAAiAtkyidDWA', 4.584527220630371),
 ('PgPw-6YU3Hny5nQx2qkzUg', 4.571108622620381)]